In [1]:
import pandas as pd

In [2]:
box_office = pd.read_csv("../analysis/boxoffice_data.csv")
movie_info = pd.read_csv("../analysis/movie_info_data.csv")
distributor_labels = pd.read_csv("../analysis/distributor_labels.csv")
movie_genre_multi_hot = pd.read_csv("../analysis/movie_genre_multi_hot.csv")
movie_summary_embedding = pd.read_csv("../analysis/movie_summary_embedding.csv")

# 1. 이상치 영화 제거

산상수훈, 더 퍼스트 슬램덩크

In [3]:
box_office = box_office[~box_office['Movie_Title'].isin(['산상수훈', '더 퍼스트 슬램덩크'])]
box_office.head()

,Movie_Title,Release_Date,Audience_Count,Show_Count,Week,Date,Holiday,Moonhwa,Pandemic,now_showing,Year
0,#살아있다,2020-06-24,204066,7223,1,2020-06-24,0,1,1,0,2020
1,#살아있다,2020-06-24,148991,7311,1,2020-06-25,0,0,1,0,2020
2,#살아있다,2020-06-24,152550,7839,1,2020-06-26,0,0,1,0,2020
3,#살아있다,2020-06-24,294255,9221,1,2020-06-27,1,0,1,0,2020
4,#살아있다,2020-06-24,256133,9044,1,2020-06-28,1,0,1,0,2020


In [4]:
movie_info = movie_info[~movie_info['Movie_Title'].isin(['산상수훈', '더 퍼스트 슬램덩크'])]
movie_info.head()

,Movie_Title,Main_Country,Grade,now_showing,director1,director2,director3,director4,director5,actor1,...,Distributor_6,Distributor_7,Distributor_8,genre1,genre2,genre3,genre4,genre5,genre6,genre7
0,#살아있다,한국,15세이상관람가,0,조일형,NaN,NaN,NaN,NaN,박신혜,...,NaN,NaN,NaN,드라마,NaN,NaN,NaN,NaN,NaN,NaN
1,#아이엠히어,프랑스,12세이상관람가,0,에릭 라티고,NaN,NaN,NaN,NaN,빈센트 네메스,...,NaN,NaN,NaN,멜로/로맨스,코미디,NaN,NaN,NaN,NaN,NaN
2,#위왓치유,기타,청소년관람불가,0,바르보라 차르포바,비트 클루삭,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,다큐멘터리,NaN,NaN,NaN,NaN,NaN,NaN
3,(여자)아이들 월드투어 [아이돌] 인 시네마,한국,12세이상관람가,0,오윤동,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,공연,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0MHz,한국,15세이상관람가,0,유선동,NaN,NaN,NaN,NaN,박명신,...,NaN,NaN,NaN,공포(호러),NaN,NaN,NaN,NaN,NaN,NaN


# 2. 총 관객수, 상영일자, 상영주차 - 종속 변수

In [5]:
box_office.head()

,Movie_Title,Release_Date,Audience_Count,Show_Count,Week,Date,Holiday,Moonhwa,Pandemic,now_showing,Year
0,#살아있다,2020-06-24,204066,7223,1,2020-06-24,0,1,1,0,2020
1,#살아있다,2020-06-24,148991,7311,1,2020-06-25,0,0,1,0,2020
2,#살아있다,2020-06-24,152550,7839,1,2020-06-26,0,0,1,0,2020
3,#살아있다,2020-06-24,294255,9221,1,2020-06-27,1,0,1,0,2020
4,#살아있다,2020-06-24,256133,9044,1,2020-06-28,1,0,1,0,2020


In [6]:
temp1 = box_office.groupby("Movie_Title")["Audience_Count"].sum().reset_index().rename(columns={"Audience_Count":"Total_Audience_Count"})
temp2 = box_office.groupby("Movie_Title")["Date"].nunique().reset_index().rename(columns={"Date":"Total_Show_Days"})
result = pd.merge(temp1, temp2, on="Movie_Title")

In [7]:
result.sort_values(by="Total_Show_Days", ascending=False)

,Movie_Title,Total_Audience_Count,Total_Show_Days
446,극장판 주술회전 0,656866,402
2473,스즈메의 문단속,5516366,345
3024,에브리씽 에브리웨어 올 앳 원스,383743,338
4624,헤어질 결심,1887567,281
483,기생충,10303224,278
...,...,...,...
2657,써클,198,2
327,궁,7,2
2758,아웃사이더,300,2
509,깝스,103,2


In [8]:
temp = box_office.groupby("Movie_Title")["Week"].max().reset_index().rename(columns={"Week":"Total_Weeks"})
result = pd.merge(result, temp, on="Movie_Title")

In [9]:
result["Total_Weeks"].describe()

count    4738.000000
mean        7.676024
std         4.871966
min         2.000000
25%         5.000000
50%         7.000000
75%         9.000000
max        62.000000
Name: Total_Weeks, dtype: float64

# 1,2주차 평일, 공휴일

In [10]:
temp_box = box_office[box_office["Movie_Title"].isin(result["Movie_Title"])].copy()
temp_box = temp_box[temp_box["Week"].isin([1,2])]



agg = {}

# 1주차
wk1 = temp_box[temp_box["Week"] == 1].groupby("Movie_Title")
agg["wk1_Audience"] = wk1["Audience_Count"].sum() #1주차 관객수
agg["wk1_AudiencePerShow"] = wk1["Audience_Count"].sum() / wk1["Show_Count"].sum() #1주차 상영횟수당 관객수

# 2주차
wk2 = temp_box[temp_box["Week"] == 2].groupby("Movie_Title")
agg["wk2_Audience"] = wk2["Audience_Count"].sum() #2주차 관객수
agg["wk2_AudiencePerShow"] = wk2["Audience_Count"].sum() / wk2["Show_Count"].sum() #2주차 상영횟수당 관객수

# 1, 2주차 상영횟수 변화율
agg["Show_Change"] = wk2["Show_Count"].sum() / wk1["Show_Count"].sum()


# # 초반 평일 유지율
# wk1_wd = temp_box[(temp_box["Week"] == 1) & (temp_box["Holiday"] == 0)].groupby("Movie_Title") #1주차 평일 일평균 관객수
# wk2_wd = temp_box[(temp_box["Week"] == 2) & (temp_box["Holiday"] == 0)].groupby("Movie_Title") #2주차 평일 일평균 관객수
# agg["opening_Wd_Retention"] = (wk2_wd["Audience_Count"].sum() / wk2_wd["Audience_Count"].count()) / \
#                              (wk1_wd["Audience_Count"].sum() / wk1_wd["Audience_Count"].count())
# 만약 영화가 1주차 주말에 개봉한다면 해당 값은 NaN이 되므로 이것을 사용하지 않는다



# 초반 휴일 유지율
wk1_ho = temp_box[(temp_box["Week"] == 1) & (temp_box["Holiday"] == 1)].groupby("Movie_Title") #1주차 휴일 일평균 관객수
wk2_ho = temp_box[(temp_box["Week"] == 2) & (temp_box["Holiday"] == 1)].groupby("Movie_Title") #2주차 휴일 일평균 관객수
agg["opening_Ho_Retention"] = (wk2_ho["Audience_Count"].sum() / wk2_ho["Audience_Count"].count()) / \
                             (wk1_ho["Audience_Count"].sum() / wk1_ho["Audience_Count"].count())


# 1주차 공휴일 평균 관객수 및 평균 상영횟수
agg["wk1_Holiday_AudienceMean"] = wk1_ho["Audience_Count"].mean()
agg["wk1_Holiday_ShowMean"] = wk1_ho["Show_Count"].mean()
# 2주차 공휴일 평균 관객수 및 평균 상영횟수
agg["wk2_Holiday_AudienceMean"] = wk2_ho["Audience_Count"].mean()
agg["wk2_Holiday_ShowMean"] = wk2_ho["Show_Count"].mean()



# 2주차까지 관객수 표준편차
aud_std = temp_box[temp_box["Week"].isin([1,2])].groupby("Movie_Title")["Audience_Count"].std()
agg["opening_AudienceStd"] = aud_std

# 결과 데이터프레임
opening_stats = pd.DataFrame(agg)
opening_stats.reset_index(inplace=True)
opening_stats.head()

,Movie_Title,wk1_Audience,wk1_AudiencePerShow,wk2_Audience,wk2_AudiencePerShow,Show_Change,opening_Ho_Retention,wk1_Holiday_AudienceMean,wk1_Holiday_ShowMean,wk2_Holiday_AudienceMean,wk2_Holiday_ShowMean,opening_AudienceStd
0,#살아있다,1055995.0,25.985408,486928,10.470444,1.144372,0.370640,275194.0,9132.5,101998.000000,7130.500000,84382.617449
1,#아이엠히어,11928.0,4.201479,3606,2.366142,0.536809,0.113489,3714.0,760.0,421.500000,121.000000,1354.388188
2,#위왓치유,538.0,3.183432,453,2.849057,0.940828,0.480000,162.5,43.0,78.000000,18.500000,44.378947
3,(여자)아이들 월드투어 [아이돌] 인 시네마,2380.0,5.905707,1347,5.659664,0.590571,0.529781,478.5,73.5,253.500000,31.000000,200.865155
4,0.0MHz,104705.0,27.496061,26441,14.963780,0.464023,0.110299,23868.5,653.5,2632.666667,94.666667,10237.742079


In [11]:
opening_stats.isna().sum()

Movie_Title                   0
wk1_Audience                  5
wk1_AudiencePerShow           5
wk2_Audience                  0
wk2_AudiencePerShow           0
Show_Change                   5
opening_Ho_Retention        192
wk1_Holiday_AudienceMean     25
wk1_Holiday_ShowMean         25
wk2_Holiday_AudienceMean    184
wk2_Holiday_ShowMean        184
opening_AudienceStd           0
dtype: int64

In [12]:
opening_stats[opening_stats.isna().any(axis=1)].reset_index(drop=True)

,Movie_Title,wk1_Audience,wk1_AudiencePerShow,wk2_Audience,wk2_AudiencePerShow,Show_Change,opening_Ho_Retention,wk1_Holiday_AudienceMean,wk1_Holiday_ShowMean,wk2_Holiday_AudienceMean,wk2_Holiday_ShowMean,opening_AudienceStd
0,22년 후의 고백,677.0,3.208531,358,4.211765,0.402844,NaN,149.0,36.5,NaN,NaN,50.857644
1,3096일,192.0,6.857143,46,7.666667,0.214286,NaN,50.5,2.0,NaN,NaN,20.500407
2,5 투 7,2695.0,7.054974,1666,7.677419,0.568063,NaN,785.5,91.5,NaN,NaN,178.917113
3,EBS친구들 씽씽 댄스파티,22.0,22.000000,38,38.000000,1.000000,NaN,NaN,NaN,NaN,NaN,11.313708
4,K스쿨,310.0,4.189189,42,2.470588,0.229730,NaN,34.0,10.5,NaN,NaN,46.956413
...,...,...,...,...,...,...,...,...,...,...,...,...
187,호두까기 인형,186.0,186.000000,186,186.000000,1.000000,NaN,186.0,1.0,NaN,NaN,0.000000
188,호라이즌 라인,1459.0,3.449173,339,2.421429,0.330969,NaN,290.0,83.0,NaN,NaN,131.606146
189,호미사이드,1403.0,48.379310,257,25.700000,0.344828,NaN,456.0,7.5,NaN,NaN,188.467504
190,호스틸,325.0,3.532609,81,2.531250,0.347826,NaN,84.5,20.0,NaN,NaN,34.283135


결측치가 있는건 전부 제거해도 괜찮을 것 같다

평일보단 공휴일이 관객수가 더 많으므로 공휴일에 집중하는게 좋을 것 같다  
따라서 초반_평일유지율 변수를 제거하고 대신 1,2주차 공휴일의 평균 관객수, 상영횟수를 추가하였다

In [13]:
opening_stats.dropna(inplace=True)

In [14]:
result = result.merge(opening_stats, on="Movie_Title", how="inner")

In [15]:
result.head()

,Movie_Title,Total_Audience_Count,Total_Show_Days,Total_Weeks,wk1_Audience,wk1_AudiencePerShow,wk2_Audience,wk2_AudiencePerShow,Show_Change,opening_Ho_Retention,wk1_Holiday_AudienceMean,wk1_Holiday_ShowMean,wk2_Holiday_AudienceMean,wk2_Holiday_ShowMean,opening_AudienceStd
0,#살아있다,1899697,62,10,1055995.0,25.985408,486928,10.470444,1.144372,0.370640,275194.0,9132.5,101998.000000,7130.500000,84382.617449
1,#아이엠히어,16271,27,5,11928.0,4.201479,3606,2.366142,0.536809,0.113489,3714.0,760.0,421.500000,121.000000,1354.388188
2,#위왓치유,1314,35,6,538.0,3.183432,453,2.849057,0.940828,0.480000,162.5,43.0,78.000000,18.500000,44.378947
3,(여자)아이들 월드투어 [아이돌] 인 시네마,3896,14,3,2380.0,5.905707,1347,5.659664,0.590571,0.529781,478.5,73.5,253.500000,31.000000,200.865155
4,0.0MHz,134918,28,5,104705.0,27.496061,26441,14.963780,0.464023,0.110299,23868.5,653.5,2632.666667,94.666667,10237.742079


In [16]:
result.shape

(4546, 15)

# 팬데믹, 개봉연도, 개봉월

In [17]:
box_office.head()

,Movie_Title,Release_Date,Audience_Count,Show_Count,Week,Date,Holiday,Moonhwa,Pandemic,now_showing,Year
0,#살아있다,2020-06-24,204066,7223,1,2020-06-24,0,1,1,0,2020
1,#살아있다,2020-06-24,148991,7311,1,2020-06-25,0,0,1,0,2020
2,#살아있다,2020-06-24,152550,7839,1,2020-06-26,0,0,1,0,2020
3,#살아있다,2020-06-24,294255,9221,1,2020-06-27,1,0,1,0,2020
4,#살아있다,2020-06-24,256133,9044,1,2020-06-28,1,0,1,0,2020


In [18]:
temp_month = box_office[["Movie_Title", "Release_Date"]].drop_duplicates().reset_index(drop=True)
temp_month['Year'] = pd.to_datetime(temp_month['Release_Date']).dt.year
temp_month['Month'] = pd.to_datetime(temp_month['Release_Date']).dt.month
temp_month[['Movie_Title', 'Release_Date', "Year", 'Month']].head()

,Movie_Title,Release_Date,Year,Month
0,#살아있다,2020-06-24,2020,6
1,#아이엠히어,2021-01-14,2021,1
2,#위왓치유,2021-06-03,2021,6
3,(여자)아이들 월드투어 [아이돌] 인 시네마,2025-01-08,2025,1
4,0.0MHz,2019-05-29,2019,5


In [19]:
result = result.merge(temp_month[['Movie_Title', "Year", 'Month']], on="Movie_Title", how="inner")

In [20]:
result.shape

(4546, 17)

In [21]:
result.head()

,Movie_Title,Total_Audience_Count,Total_Show_Days,Total_Weeks,wk1_Audience,wk1_AudiencePerShow,wk2_Audience,wk2_AudiencePerShow,Show_Change,opening_Ho_Retention,wk1_Holiday_AudienceMean,wk1_Holiday_ShowMean,wk2_Holiday_AudienceMean,wk2_Holiday_ShowMean,opening_AudienceStd,Year,Month
0,#살아있다,1899697,62,10,1055995.0,25.985408,486928,10.470444,1.144372,0.370640,275194.0,9132.5,101998.000000,7130.500000,84382.617449,2020,6
1,#아이엠히어,16271,27,5,11928.0,4.201479,3606,2.366142,0.536809,0.113489,3714.0,760.0,421.500000,121.000000,1354.388188,2021,1
2,#위왓치유,1314,35,6,538.0,3.183432,453,2.849057,0.940828,0.480000,162.5,43.0,78.000000,18.500000,44.378947,2021,6
3,(여자)아이들 월드투어 [아이돌] 인 시네마,3896,14,3,2380.0,5.905707,1347,5.659664,0.590571,0.529781,478.5,73.5,253.500000,31.000000,200.865155,2025,1
4,0.0MHz,134918,28,5,104705.0,27.496061,26441,14.963780,0.464023,0.110299,23868.5,653.5,2632.666667,94.666667,10237.742079,2019,5


In [22]:
first_pandemic = box_office.groupby('Movie_Title')['Pandemic'].first().reset_index()
first_pandemic.head()

,Movie_Title,Pandemic
0,#살아있다,1
1,#아이엠히어,1
2,#위왓치유,1
3,(여자)아이들 월드투어 [아이돌] 인 시네마,2
4,0.0MHz,0


In [23]:
result = result.merge(first_pandemic, on='Movie_Title', how='inner')

In [24]:
result.shape

(4546, 18)

# 배급사 멀티핫 인코딩

In [25]:
distributor_labels.head()

,Distributor,label
0,씨제이엔터테인먼트,대형/흥행
1,유니버설픽쳐스인터내셔널 코리아(유),대형/흥행
2,(주)영화사 진진,대형/비흥행
3,(주)씨제이이엔엠,대형/흥행
4,롯데쇼핑㈜롯데엔터테인먼트,대형/흥행


In [26]:
distributor_labels["label"].unique()

array(['대형/흥행', '대형/비흥행', '소형/비흥행', '소형/흥행'], dtype=object)

In [27]:
distributor_labels.replace({"label": {"대형/흥행": "big_hit", "대형/비흥행": "big_flop", 
                                      "소형/비흥행": "small_flop", "소형/흥행": "small_hit"}}, inplace=True)
distributor_labels.head()

,Distributor,label
0,씨제이엔터테인먼트,big_hit
1,유니버설픽쳐스인터내셔널 코리아(유),big_hit
2,(주)영화사 진진,big_flop
3,(주)씨제이이엔엠,big_hit
4,롯데쇼핑㈜롯데엔터테인먼트,big_hit


In [28]:
distributor_label_dummies = pd.get_dummies(distributor_labels['label'], prefix='label').astype(int)
distributor_labels_onehot = pd.concat([distributor_labels[['Distributor']], distributor_label_dummies], axis=1)
distributor_labels_onehot.head()

,Distributor,label_big_flop,label_big_hit,label_small_flop,label_small_hit
0,씨제이엔터테인먼트,0,1,0,0
1,유니버설픽쳐스인터내셔널 코리아(유),0,1,0,0
2,(주)영화사 진진,1,0,0,0
3,(주)씨제이이엔엠,0,1,0,0
4,롯데쇼핑㈜롯데엔터테인먼트,0,1,0,0


In [29]:
movie_info[['Movie_Title', 'Distributor_1', 'Distributor_2', 'Distributor_3', 'Distributor_4',
       'Distributor_5', 'Distributor_6', 'Distributor_7', 'Distributor_8']].head()

,Movie_Title,Distributor_1,Distributor_2,Distributor_3,Distributor_4,Distributor_5,Distributor_6,Distributor_7,Distributor_8
0,#살아있다,롯데컬처웍스(주)롯데엔터테인먼트,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,#아이엠히어,(주)넥스트엔터테인먼트월드(NEW),NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,#위왓치유,찬란,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,(여자)아이들 월드투어 [아이돌] 인 시네마,씨제이포디플렉스 주식회사,씨지브이 아이스콘,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0MHz,(주)스마일이엔티,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
dist_cols = [f'Distributor_{i}' for i in range(1, 9)]
movies_long = movie_info.melt(
    id_vars=['Movie_Title'],
    value_vars=dist_cols,
    value_name='Distributor'
)
movies_long.head()

,Movie_Title,variable,Distributor
0,#살아있다,Distributor_1,롯데컬처웍스(주)롯데엔터테인먼트
1,#아이엠히어,Distributor_1,(주)넥스트엔터테인먼트월드(NEW)
2,#위왓치유,Distributor_1,찬란
3,(여자)아이들 월드투어 [아이돌] 인 시네마,Distributor_1,씨제이포디플렉스 주식회사
4,0.0MHz,Distributor_1,(주)스마일이엔티


In [31]:
movies_long = movies_long.dropna(subset=['Distributor'])
movies_long = movies_long.drop(columns=['variable'])
movies_long.head()

,Movie_Title,Distributor
0,#살아있다,롯데컬처웍스(주)롯데엔터테인먼트
1,#아이엠히어,(주)넥스트엔터테인먼트월드(NEW)
2,#위왓치유,찬란
3,(여자)아이들 월드투어 [아이돌] 인 시네마,씨제이포디플렉스 주식회사
4,0.0MHz,(주)스마일이엔티


In [32]:
merged_df = pd.merge(
    movies_long,
    distributor_labels_onehot,
    on='Distributor',
    how='left'
)
merged_df.tail()

,Movie_Title,Distributor,label_big_flop,label_big_hit,label_small_flop,label_small_hit
5436,원더,그린나래미디어(주),1.0,0.0,0.0,0.0
5437,클로저,소니픽쳐스릴리징코리아,0.0,1.0,0.0,0.0
5438,"그 시절, 우리가 좋아했던 소녀",씨너스엔터테인먼트(주),0.0,1.0,0.0,0.0
5439,"그 시절, 우리가 좋아했던 소녀",(주)미로비젼,1.0,0.0,0.0,0.0
5440,"그 시절, 우리가 좋아했던 소녀",(주)박수엔터테인먼트,1.0,0.0,0.0,0.0


In [33]:
merged_df.query("Movie_Title=='그 시절, 우리가 좋아했던 소녀'")

,Movie_Title,Distributor,label_big_flop,label_big_hit,label_small_flop,label_small_hit
351,"그 시절, 우리가 좋아했던 소녀",(주)제이앤씨미디어그룹,0.0,1.0,0.0,0.0
4790,"그 시절, 우리가 좋아했던 소녀",와이드 릴리즈(주),1.0,0.0,0.0,0.0
5351,"그 시절, 우리가 좋아했던 소녀",(주)이놀미디어,1.0,0.0,0.0,0.0
5423,"그 시절, 우리가 좋아했던 소녀",(주)드림팩트엔터테인먼트,1.0,0.0,0.0,0.0
5435,"그 시절, 우리가 좋아했던 소녀",(주)미로스페이스,0.0,0.0,1.0,0.0
5438,"그 시절, 우리가 좋아했던 소녀",씨너스엔터테인먼트(주),0.0,1.0,0.0,0.0
5439,"그 시절, 우리가 좋아했던 소녀",(주)미로비젼,1.0,0.0,0.0,0.0
5440,"그 시절, 우리가 좋아했던 소녀",(주)박수엔터테인먼트,1.0,0.0,0.0,0.0


In [34]:
final_movie_labels = merged_df.groupby('Movie_Title').sum().reset_index()
final_movie_labels.query("Movie_Title=='그 시절, 우리가 좋아했던 소녀'")

,Movie_Title,Distributor,label_big_flop,label_big_hit,label_small_flop,label_small_hit
351,"그 시절, 우리가 좋아했던 소녀",(주)제이앤씨미디어그룹와이드 릴리즈(주)(주)이놀미디어(주)드림팩트엔터테인먼트(주)...,5.0,2.0,1.0,0.0


In [35]:
result = result.merge(final_movie_labels.drop(columns=["Distributor"]), on='Movie_Title', how='left')

In [36]:
result.rename(columns={"label_big_hit":"dist_big_hit", "label_big_flop":"dist_big_flop",
                       "label_small_flop":"dist_small_flop", "label_small_hit":"dist_small_hit"}, inplace=True)

In [37]:
result.shape

(4546, 22)

In [38]:
result.isna().sum()

Movie_Title                 0
Total_Audience_Count        0
Total_Show_Days             0
Total_Weeks                 0
wk1_Audience                0
wk1_AudiencePerShow         0
wk2_Audience                0
wk2_AudiencePerShow         0
Show_Change                 0
opening_Ho_Retention        0
wk1_Holiday_AudienceMean    0
wk1_Holiday_ShowMean        0
wk2_Holiday_AudienceMean    0
wk2_Holiday_ShowMean        0
opening_AudienceStd         0
Year                        0
Month                       0
Pandemic                    0
dist_big_flop               0
dist_big_hit                0
dist_small_flop             0
dist_small_hit              0
dtype: int64

In [39]:
result.head()

,Movie_Title,Total_Audience_Count,Total_Show_Days,Total_Weeks,wk1_Audience,wk1_AudiencePerShow,wk2_Audience,wk2_AudiencePerShow,Show_Change,opening_Ho_Retention,...,wk2_Holiday_AudienceMean,wk2_Holiday_ShowMean,opening_AudienceStd,Year,Month,Pandemic,dist_big_flop,dist_big_hit,dist_small_flop,dist_small_hit
0,#살아있다,1899697,62,10,1055995.0,25.985408,486928,10.470444,1.144372,0.370640,...,101998.000000,7130.500000,84382.617449,2020,6,1,0.0,1.0,0.0,0.0
1,#아이엠히어,16271,27,5,11928.0,4.201479,3606,2.366142,0.536809,0.113489,...,421.500000,121.000000,1354.388188,2021,1,1,0.0,1.0,0.0,0.0
2,#위왓치유,1314,35,6,538.0,3.183432,453,2.849057,0.940828,0.480000,...,78.000000,18.500000,44.378947,2021,6,1,1.0,0.0,0.0,0.0
3,(여자)아이들 월드투어 [아이돌] 인 시네마,3896,14,3,2380.0,5.905707,1347,5.659664,0.590571,0.529781,...,253.500000,31.000000,200.865155,2025,1,2,1.0,1.0,0.0,0.0
4,0.0MHz,134918,28,5,104705.0,27.496061,26441,14.963780,0.464023,0.110299,...,2632.666667,94.666667,10237.742079,2019,5,0,0.0,1.0,0.0,0.0


# 영화 등급 범주형 변수

In [40]:
result = result.merge(movie_info[["Movie_Title", "Grade"]], on='Movie_Title', how='inner')

In [41]:
result.shape

(4546, 23)

# 영화 대표국적 범주형 변수

한국, 일본, 미국, 나머지는 전부 기타로 변환

In [42]:
temp_country = movie_info[['Movie_Title', 'Main_Country']].copy()

In [43]:
temp_country["Main_Country"] = temp_country["Main_Country"].apply(
    lambda x: x if x in ["한국", "미국", "일본"] else "기타"
)
temp_country["Main_Country"].unique()

array(['한국', '기타', '미국', '일본'], dtype=object)

In [44]:
temp_country.head()

,Movie_Title,Main_Country
0,#살아있다,한국
1,#아이엠히어,기타
2,#위왓치유,기타
3,(여자)아이들 월드투어 [아이돌] 인 시네마,한국
4,0.0MHz,한국


In [45]:
result = result.merge(temp_country, on='Movie_Title', how='inner')

In [46]:
result.shape

(4546, 24)

# 현재 상영중인 영화

In [47]:
result = result.merge(movie_info[["Movie_Title", "now_showing"]], on='Movie_Title', how='inner')

In [48]:
result.shape

(4546, 25)

In [49]:
result.columns

Index(['Movie_Title', 'Total_Audience_Count', 'Total_Show_Days', 'Total_Weeks',
       'wk1_Audience', 'wk1_AudiencePerShow', 'wk2_Audience',
       'wk2_AudiencePerShow', 'Show_Change', 'opening_Ho_Retention',
       'wk1_Holiday_AudienceMean', 'wk1_Holiday_ShowMean',
       'wk2_Holiday_AudienceMean', 'wk2_Holiday_ShowMean',
       'opening_AudienceStd', 'Year', 'Month', 'Pandemic', 'dist_big_flop',
       'dist_big_hit', 'dist_small_flop', 'dist_small_hit', 'Grade',
       'Main_Country', 'now_showing'],
      dtype='object')

# 영화 줄거리 임베딩

In [50]:
movie_summary_embedding.head()

,Movie_Title,summary_256,summary_128,summary_64
0,#살아있다,"[0.00747680664, -0.139648438, 0.064453125, 0.0...","[0.00747680664, -0.139648438, 0.064453125, 0.0...","[0.0074768066, -0.13964844, 0.064453125, 0.058..."
1,#아이엠히어,"[-0.0153198242, -0.1640625, 0.0505371094, 0.05...","[-0.01531982, -0.1640625, 0.05053711, 0.058349...","[-0.01531982, -0.1640625, 0.05053711, 0.058349..."
2,#위왓치유,"[0.0668945312, -0.151367188, 0.125976562, 0.06...","[0.06689453, -0.15136719, 0.12597656, 0.062988...","[0.06689453, -0.15136719, 0.12597656, 0.062988..."
3,(여자)아이들 월드투어 [아이돌] 인 시네마,"[0.064453125, -0.122558594, 0.0571289062, 0.07...","[0.064453125, -0.122558594, 0.0571289062, 0.07...","[0.06445312, -0.12255859, 0.05712891, 0.076660..."
4,0.0MHz,"[0.1015625, -0.125, 0.10791016, 0.07177734, 0....","[0.1015625, -0.125, 0.10791016, 0.07177734, 0....","[0.1015625, -0.125, 0.10791016, 0.07177734, 0...."


In [51]:
import pandas as pd
import ast

# 'summary_256' 컬럼의 각 요소가 문자열('[0.007..., -0.139..., ...]') 형태라고 가정합니다.
# ast.literal_eval로 문자열을 파이썬 리스트 객체로 안전하게 변환합니다.
embedding_list = movie_summary_embedding['summary_256'].apply(ast.literal_eval).tolist()

# 2차원 리스트를 사용하여 DataFrame 생성
summary_256_df = pd.DataFrame(
    embedding_list, 
    columns=[f'e{i+1}' for i in range(256)]
)

summary_256_df['Movie_Title'] = movie_summary_embedding['Movie_Title']

In [52]:
summary_256_df.head()

,e1,e2,e3,e4,e5,e6,e7,e8,e9,e10,...,e248,e249,e250,e251,e252,e253,e254,e255,e256,Movie_Title
0,0.007477,-0.139648,0.064453,0.058594,0.040771,-0.062012,-0.027710,0.136719,-0.121582,-0.085938,...,-0.009949,-0.013977,-0.018799,-0.028809,-0.027710,-0.047607,0.021973,0.031738,0.033203,#살아있다
1,-0.015320,-0.164062,0.050537,0.058350,0.103027,0.017822,-0.083008,0.167969,-0.044678,-0.098145,...,-0.039062,-0.021362,-0.030151,-0.024536,-0.025757,0.011536,-0.006256,0.063965,0.065430,#아이엠히어
2,0.066895,-0.151367,0.125977,0.062988,0.043457,-0.005249,-0.019287,0.124023,-0.070312,-0.090332,...,-0.039551,-0.032715,0.033447,-0.027588,-0.070312,-0.003204,0.004242,0.034668,0.022949,#위왓치유
3,0.064453,-0.122559,0.057129,0.076660,0.053467,-0.085938,-0.066895,-0.005646,-0.059082,-0.094238,...,-0.069824,-0.027832,-0.009705,-0.031738,-0.043701,-0.003937,0.022827,-0.000275,0.004150,(여자)아이들 월드투어 [아이돌] 인 시네마
4,0.101562,-0.125000,0.107910,0.071777,0.088867,-0.076660,-0.042969,0.083984,-0.114258,-0.060791,...,-0.034424,-0.023926,0.010681,-0.054443,-0.018311,-0.010986,0.011353,0.018555,0.026611,0.0MHz


In [53]:
summary_256_df.shape

(4740, 257)

In [54]:
result = result.merge(summary_256_df, on="Movie_Title", how="inner")

In [55]:
result.shape

(4546, 281)

# 영화 카테고리

# 현재 상영중인 영화 분리

In [56]:
now_result = result[result["now_showing"] == 1] #현재 상영중인 영화
result = result[result["now_showing"] == 0] #상영이 종료된 영화

In [57]:
print(result.shape)
print(now_result.shape)

(4517, 281)
(29, 281)


In [58]:
now_result.head()

,Movie_Title,Total_Audience_Count,Total_Show_Days,Total_Weeks,wk1_Audience,wk1_AudiencePerShow,wk2_Audience,wk2_AudiencePerShow,Show_Change,opening_Ho_Retention,...,e247,e248,e249,e250,e251,e252,e253,e254,e255,e256
94,F1 더 무비,4878215,74,11,478085.0,18.453893,498575,21.791818,0.883120,0.919323,...,-0.000246,-0.020752,0.010559,-0.007599,-0.025391,0.008606,-0.003235,0.008972,0.028931,0.066406
150,강령: 귀신놀이,71395,32,5,41447.0,20.733867,25043,14.013990,0.893947,0.426405,...,0.004852,-0.045166,-0.014221,0.001099,-0.017456,-0.019287,-0.007996,-0.032715,0.034668,0.003586
410,극장판 귀멸의 칼날: 무한성편,3751895,16,3,1623200.0,63.005085,1504807,30.722887,1.901176,0.575226,...,-0.007050,-0.024902,-0.024536,0.004028,-0.035645,0.007935,-0.028687,0.023193,0.028076,-0.001259
475,긴키 지방의 어느 장소에 대하여,265588,25,4,101649.0,32.506876,86929,20.677688,1.344420,0.766366,...,-0.014587,-0.011475,-0.038818,0.008850,-0.039551,-0.001350,0.004028,-0.023438,0.036621,0.013672
494,꼬마마법사 주니토니,54961,41,7,11403.0,22.625000,23195,21.697848,2.121032,0.638604,...,0.013184,-0.046875,0.000866,0.047119,-0.024902,-0.006378,0.009399,0.004211,0.004547,0.028320


In [59]:
result.drop(columns=["now_showing"], inplace=True)
now_result.drop(columns=["now_showing"], inplace=True)

# 모델 데이터 - 총 관객수 1만명 이상 500만명 이하만

In [60]:
below_10k = result[result["Total_Audience_Count"] < 10_000]
upper_5m = result[result["Total_Audience_Count"] > 5_000_000]
result = result[(result["Total_Audience_Count"] >= 10_000) & (result["Total_Audience_Count"] <= 5_000_000)]

In [61]:
print(below_10k.shape, result.shape, upper_5m.shape)

(1876, 280) (2568, 280) (73, 280)


# train, test 분리

총 관람객수를 기준으로 계층적 분리  
10분위로 나누고, 각 분위마다 동일한 비율로 샘플링하여 치우침이 없게 함

In [62]:
from sklearn.model_selection import train_test_split

# stratify 기준 컬럼 선택 (예: 'Total_Audience_Count'을 구간화하여 분포 유지)
stratify_col = pd.qcut(result['Total_Audience_Count'], q=10, duplicates='drop')

train_df, test_df = train_test_split(
    result,
    test_size=0.2,
    random_state=42,
    stratify=stratify_col
)

print(train_df.shape, test_df.shape)

(2054, 280) (514, 280)


# 저장

독립변수는 실제 데이터 예측에서 제거한다

In [63]:
now_result.drop(columns=['Total_Audience_Count', "Total_Show_Days", "Total_Weeks"], inplace=True)

In [64]:
train_df.to_csv("model_train_data.csv", index=False, encoding='utf-8-sig')
test_df.to_csv("model_test_data.csv", index=False, encoding='utf-8-sig')
now_result.to_csv("model_predict_data.csv", index=False, encoding='utf-8-sig')
below_10k.to_csv("model_below_10k_data.csv", index=False, encoding='utf-8-sig')
upper_5m.to_csv("model_upper_5M_data.csv", index=False, encoding='utf-8-sig')